# 概述

k-近邻算法可以完成很多分类任务，但是最大的缺点就是无法给出数据内在含义，决策树的主要优势就在于数据形式非常容易理解

优点：计算复杂度不高，输出结果易于理解，对中间值的缺失不敏感，可以处理不相关特征数据  
缺点：可能会产生过度匹配问题  
适用数据类型：数值型和标称型

# 决策树的构造

## 信息增益

在划分数据集之前之后信息发生的变化成为信息增益  
计算每个特征值划分数据集获得的信息增益，选择获得信息增益最高的特征

符号$x_i$的信息定义为  
$$l(x_i)=-log_2p(x_i)$$  
其中$p(x_i)$是选择该分类的概率  
  
信息期望值（熵）为  
$$H=-\sum_{i=1}^{n}{p(x_i)log_2p(x_i)}$$

In [7]:
from math import log
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0
    for key in labelCounts:
        prob = labelCounts[key] / numEntries
        #print("标签%s的分类概率为：%s" % (key, prob))
        #print("信息增益为：%s" % (-log(prob, 2)))
        shannonEnt -= prob*log(prob, 2)
    return shannonEnt

In [8]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

myDat, labels = createDataSet()
calcShannonEnt(myDat)

0.9709505944546686

熵越高，则混合的数据也越多  

In [15]:
myDat[0][-1] = 'maybe'
print(myDat)
calcShannonEnt(myDat) # 增加了分类，熵的值变大，越无序

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]


1.3709505944546687

另一个度量集合无序程度的方法是基尼不纯度（Gini impurity），简单的说就是从一个数据集种随机选取子项，度量其被错误分类到其他分组里的概率。

## 划分数据集

In [19]:
# dataSet:待划分的数据集； axis:划分数据集的特征； value:需要返回特征的值
# 讲符合特征axis=value的数据集dataSet取出来，剔除axis这个字段重组成列表存储
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reduceFeatVec = featVec[:axis]
            reduceFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reduceFeatVec)
    return retDataSet

In [18]:
myDat, labels = createDataSet()
print(myDat)
splitDataSet(myDat, 0, 1)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]


[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [22]:
calcShannonEnt(dataSet)

0.9709505944546686

In [20]:
# def chooseBestFeatureToSplit(dataSet):
dataSet = myDat.copy()
numFeatures = len(dataSet[0]) - 1
baseEntropy = calcShannonEnt(dataSet)
bestInfoGain = 0
beatFeature = -1
for i in range(numFeatures):
    featList = [example[i] for example in dataSet] #把一列的特征值取了出来
    uniqueVals = set(featList)
    newEntropy = 0
    for value in uniqueVals:
        subDataSet = splitDataSet(dataSet, i, value)
        prob = len(subDataSet) / len(dataSet)
        newEntropy += prob * calcShannonEnt(subDataSet)
